# Non Graded  Challenge 3
---
> Arief Luqmannurhakim
>
> Batch 001 -- FTDS

# Load dataframe
---

In [1]:
import sqlite3
import pandas as pd

In [2]:
!printf '%s\n' *

assessments.csv
courses.csv
hasil_record_data.xlsx
sample_data
studentAssessment.csv.zip
studentInfo.csv.zip
studentRegistration.csv.zip
vle.csv


In [3]:
list_data = ['courses.csv',
'studentAssessment.csv.zip',
'studentInfo.csv.zip',
'studentRegistration.csv.zip',
'vle.csv']

df = []

for i in list_data:
  if i[-1] == 'p' :
    df.append(pd.read_csv(i, compression='zip'))
  

  df.append(pd.read_csv(i))

In [4]:
for i in range(len(list_data)):
  print('\n\n')
  print(df[i].info())




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   code_module                 22 non-null     object
 1   code_presentation           22 non-null     object
 2   module_presentation_length  22 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 656.0+ bytes
None



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173912 entries, 0 to 173911
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id_assessment   173912 non-null  int64  
 1   id_student      173912 non-null  int64  
 2   date_submitted  173912 non-null  int64  
 3   is_banked       173912 non-null  int64  
 4   score           173739 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 6.6 MB
None



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173912 entri

In [5]:
df[3]

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass
...,...,...,...,...,...,...,...,...,...,...,...,...
32588,GGG,2014J,2640965,F,Wales,Lower Than A Level,10-20,0-35,0,30,N,Fail
32589,GGG,2014J,2645731,F,East Anglian Region,Lower Than A Level,40-50%,35-55,0,30,N,Distinction
32590,GGG,2014J,2648187,F,South Region,A Level or Equivalent,20-30%,0-35,0,30,Y,Pass
32591,GGG,2014J,2679821,F,South East Region,Lower Than A Level,90-100%,35-55,0,30,N,Withdrawn


# Create Database
---

In [6]:
table_csv = [
  'courses',
 'studentAssessment',
 'studentInfo',
 'studentRegistration',
 'vle']

In [7]:
db = sqlite3.connect('student_data.db') # Automatically creating database if doesn't exists in current directory

cur = db.cursor()
cur.execute(''' CREATE TABLE IF NOT EXISTS assessments(
  code_module text,
  code_presentation text,
  id_assessment integer NOT NULL,
  assessment_type text,
  date real,
  weight real
  
  );  ''')

cur.execute(''' INSERT INTO assessments (code_module, code_presentation, id_assessment, assessment_type, date, weight)
VALUES ('AAA', '2013J', 1752, 'TMA', 19.0, 10.0),
       ('GGG', '2014J', 37436, 'TMA', 124.0, 0.0),
       ('GGG', '2014J', 37444, 'Exam', 229.0, 100.0) ''')

cur.execute('SELECT * FROM assessments')

for x in cur.fetchall():
  print(x)

assesment_df = pd.read_csv('assessments.csv')
assesment_df.to_sql('assessments', db, if_exists='replace', index=False)

db.commit()
db.close()

('AAA', '2013J', 1752, 'TMA', 19.0, 10.0)
('GGG', '2014J', 37436, 'TMA', 124.0, 0.0)
('GGG', '2014J', 37444, 'Exam', 229.0, 100.0)


In [8]:
db = sqlite3.connect('student_data.db') # Automatically creating database if doesn't exists in current directory

for i in range(len(table_csv)):

  df[i].to_sql(table_csv[i], db, if_exists='replace', index=False)


db.commit()
db.close()

# Query database
---

In [9]:
db = sqlite3.connect('student_data.db')

df_query = pd.read_sql_query(
    ''' SELECT studentAssessment.id_student, vle.region, vle.code_presentation,  vle.code_module, assessments.assessment_type, studentAssessment.score
        FROM vle
        INNER JOIN studentAssessment ON vle.id_student = studentAssessment.id_student
        INNER JOIN assessments ON studentAssessment.id_assessment = assessments.id_assessment
        WHERE vle.code_presentation == "2014B"
        ; ''', db)
db.close()

In [10]:
df_query.head(100)

,id_student,region,code_presentation,code_module,assessment_type,score
0,25997,London Region,2014B,BBB,TMA,73.0
1,25997,London Region,2014B,BBB,CMA,100.0
2,27891,Scotland,2014B,BBB,TMA,73.0
3,27891,Scotland,2014B,BBB,TMA,67.0
4,27891,Scotland,2014B,BBB,TMA,55.0
...,...,...,...,...,...,...
95,78255,North Region,2014B,BBB,TMA,76.0
96,78255,North Region,2014B,BBB,TMA,76.0
97,78255,North Region,2014B,BBB,TMA,63.0
98,78255,North Region,2014B,BBB,CMA,100.0


In [11]:
df_query.to_excel('hasil_record_data.xlsx')